In [6]:
import pickle
import json
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from langfuse import Langfuse

load_dotenv(".env")

True

In [47]:
df = pd.read_excel("QAs/Kịch bản chat Agent nội bộ.xlsx")
df.head()

,Câu hỏi,Câu trả lời dự kiến,Tham chiếu
0,Tôi muốn hỏi về chính sách thâm niên của công ...,Thâm niên được tính từ thời điểm ký HĐLĐ chính...,NaN
1,Vậy làm việc bao nhiêu năm thì sẽ được giải th...,"CTS áp dụng chính sách 5 năm, có tính thời gia...",NaN
2,Tôi hiện tại đang làm việc tại một dự án khách...,Dựa theo chính sách mới thì từ ngày 26/08/2025...,NaN
3,"Tôi là nhân viên kinh doanh, được phép đi muộn...",Cán bộ kinh doanh được tổng cộng 10 lần đi muộ...,NaN
4,Chu kỳ đánh giá công việc tháng diễn ra khi nào?,Chu kỳ đánh giá công việc tháng được thực hiện...,NaN


In [48]:
questions = df.iloc[:, 0].tolist()
true_answers = df.iloc[:, 1].tolist()

In [9]:
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_HOST = os.getenv("LANGFUSE_HOST")
langfuse = Langfuse(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_HOST
)

In [10]:
def get_traces_data(session_id: str):
    traces = langfuse.api.trace.list(session_id=session_id)
    traces_data = traces.data[::-1]
    return traces_data

def get_observations(trace_id):
    observations = json.loads(langfuse.api.trace.get(trace_id).json())["observations"]
    generation_observations = [obs if obs["type"] == "GENERATION" else None for obs in observations]
    generation_observations = list(filter(lambda x: x is not None, generation_observations))
    return generation_observations

def get_tokens_usage_per_trace(observations):
    input_tokens = 0
    output_tokens = 0
    for obs in observations:
        input_tokens += obs["usage"]["input"]
        output_tokens += obs["usage"]["output"]
    return {
        "input_tokens": input_tokens,
        "output_tokens": output_tokens
    }

def get_tokens_usage_per_session(session_id):
    traces = get_traces_data(session_id)
    tokens_usage = {}
    for id, trace in enumerate(traces):
        observations = get_observations(trace.id)
        tokens_usage_per_trace = get_tokens_usage_per_trace(observations)
        tokens_usage_per_trace.update({"id": trace.id})
        tokens_usage[f"Trace {id}"] = tokens_usage_per_trace
    return tokens_usage
    

In [11]:
zep_session_id = "5a8e23c1-c3a7-4f5b-aff9-269fcc3f9764"
default_session_id = "8ec03156-f766-49a2-84f6-f96d0d9d64a3"

In [29]:
zep_traces_data = get_traces_data(zep_session_id)
default_traces_data = get_traces_data(default_session_id)

In [78]:
# zep_agent_answers = [json.loads(data.json())["output"].get("output") for data in zep_traces_data]
# default_agent_answers = [json.loads(data.json())["output"].get("output") for data in default_traces_data]

zep_agent_answers = []
default_agent_answers = []

for data in zep_traces_data:
    output = json.loads(data.json())["output"]
    if output.get("output"):
        zep_agent_answers.append(output.get("output"))
    else:
        print(output)
        zep_agent_answers.append(output["returnValues"].get("output"))

for data in default_traces_data:
    output = json.loads(data.json())["output"]["cmcts_agent"]["messages"][0]["kwargs"]["content"]
    default_agent_answers.append(output)

{'returnValues': {'output': '**Observation:** Sau khi kiểm tra nghiêm ngặt ngữ cảnh của từng chunk, tôi chỉ sử dụng những thông tin có độ chính xác 100% và liên quan trực tiếp đến câu hỏi để đưa ra câu trả lời bao gồm: Thông tin về thời hạn báo trước nghỉ việc theo từng loại hợp đồng từ file CTS_QT_03.\n\nKhi bạn muốn nghỉ việc tại CMC TS, **thời gian báo trước** phụ thuộc vào **loại hợp đồng lao động** bạn đang có:\n\n### Thời hạn báo trước theo loại hợp đồng:\n\n**Hợp đồng thử việc (HĐTV):**\n- **Trước 03 ngày làm việc**\n\n**Hợp đồng lao động xác định thời hạn:**\n- **Trước 30 ngày**\n\n**Hợp đồng lao động không xác định thời hạn:**\n- **Trước 45 ngày**\n\n### Quy trình nghỉ việc:\n\n**Tạo đơn trên hệ thống:**\n- Thực hiện tại mục **"Xin nghỉ việc/Seperation request"** trên hệ thống **SF4C**\n- Điền đầy đủ thông tin và **tuân thủ thời hạn báo trước**\n\n**Quy trình phê duyệt:**\n- **HĐTV**: Phê duyệt trong vòng **4 giờ làm việc** ở mỗi cấp\n- **HĐLĐ**: Phê duyệt trong vòng **2 ngày 

In [34]:
GENERAL_PURPOSE_API_URL = "https://stock.cmcts.ai/c-agent/api/v1/prediction/a1514d8f-749c-4e01-b961-811e9346d83e"

def query(payload):
    response = requests.post(GENERAL_PURPOSE_API_URL, json=payload)
    return response.json()["text"]

In [32]:
from utils import extract_section

In [56]:
print(json.dumps(json.loads(zep_traces_data[5].json()), indent=2, ensure_ascii=False))

{
  "id": "d7074ef7-167b-4ae6-8935-4d15e69d29c7",
  "timestamp": "2025-09-16T08:20:13.729000Z",
  "name": "AgentExecutor",
  "input": {
    "input": "Nếu tôi muốn nghỉ việc, cần báo trước bao lâu?"
  },
  "output": {
    "returnValues": {
      "output": "**Observation:** Sau khi kiểm tra nghiêm ngặt ngữ cảnh của từng chunk, tôi chỉ sử dụng những thông tin có độ chính xác 100% và liên quan trực tiếp đến câu hỏi để đưa ra câu trả lời bao gồm: Thông tin về thời hạn báo trước nghỉ việc theo từng loại hợp đồng từ file CTS_QT_03.\n\nKhi bạn muốn nghỉ việc tại CMC TS, **thời gian báo trước** phụ thuộc vào **loại hợp đồng lao động** bạn đang có:\n\n### Thời hạn báo trước theo loại hợp đồng:\n\n**Hợp đồng thử việc (HĐTV):**\n- **Trước 03 ngày làm việc**\n\n**Hợp đồng lao động xác định thời hạn:**\n- **Trước 30 ngày**\n\n**Hợp đồng lao động không xác định thời hạn:**\n- **Trước 45 ngày**\n\n### Quy trình nghỉ việc:\n\n**Tạo đơn trên hệ thống:**\n- Thực hiện tại mục **\"Xin nghỉ việc/Seperation 

In [33]:
with open("prompts/evaluation/human_prompt.txt", "r") as f:
    evaluate_human_prompt_template = f.read().strip()

with open("prompts/evaluation/system_prompt.txt", "r") as f:
    evaluate_system_prompt = f.read().strip()

In [80]:
zep_evaluate_results = []
for question, true_answer, zep_agent_answer in zip(questions, true_answers, zep_agent_answers):
    evaluate_human_prompt = evaluate_human_prompt_template.format(
            question=question,
            true_answer=true_answer,
            agent_answer=zep_agent_answer
        )
        
    payload = {
        "question": "Đánh giá câu trả lời từ agent so với câu trả lời chuẩn (true_answer)",
        "overrideConfig": {
                "systemMessagePrompt": evaluate_system_prompt,
                "humanMessagePrompt": evaluate_human_prompt
            }
    }

    response = query(payload)
    zep_evaluate_results.append(extract_section(response))



JSON data:
{'relevance': 10.0, 'accuracy': 10.0, 'completeness': 10.0, 'clarity': 10.0, 'tone': 10.0, 'comments': "Relevance (10.0/10.0): Câu trả lời hoàn toàn phù hợp với câu hỏi về chính sách thâm niên. Agent đã trả lời đúng trọng tâm câu hỏi về thời điểm bắt đầu tính thâm niên.\n\nAccuracy (10.0/10.0): Thông tin trong câu trả lời của agent hoàn toàn chính xác và khớp với câu trả lời chuẩn. Agent đã nêu rõ 'thâm niên được tính từ thời điểm ký Hợp đồng lao động chính thức đầu tiên'.\n\nCompleteness (10.0/10.0): Câu trả lời không chỉ cung cấp thông tin cơ bản như câu trả lời chuẩn mà còn bổ sung nhiều thông tin hữu ích khác như: chi tiết về cách tính thâm niên, thời gian không được tính vào thâm niên, và ví dụ minh họa cụ thể. Những thông tin bổ sung này rất có giá trị cho người hỏi.\n\nClarity (10.0/10.0): Câu trả lời được trình bày rất rõ ràng, có cấu trúc logic với các đề mục và gạch đầu dòng, giúp người đọc dễ dàng nắm bắt thông tin. Format phân chia rõ ràng giữa các phần thông tin

In [81]:
default_evaluate_results = []
for question, true_answer, default_agent_answer in zip(questions, true_answers, default_agent_answers):
    evaluate_human_prompt = evaluate_human_prompt_template.format(
            question=question,
            true_answer=true_answer,
            agent_answer=default_agent_answer
        )
        
    payload = {
        "question": "Đánh giá câu trả lời từ agent so với câu trả lời chuẩn (true_answer)",
        "overrideConfig": {
                "systemMessagePrompt": evaluate_system_prompt,
                "humanMessagePrompt": evaluate_human_prompt
            }
    }

    response = query(payload)
    default_evaluate_results.append(extract_section(response))



JSON data:
{'relevance': 10.0, 'accuracy': 10.0, 'completeness': 10.0, 'clarity': 10.0, 'tone': 10.0, 'comments': 'Relevance (Mức độ phù hợp): 10.0/10.0\nCâu trả lời của agent hoàn toàn phù hợp với câu hỏi về thời điểm tính thâm niên. Agent đã trả lời đúng trọng tâm câu hỏi và không đưa ra thông tin lạc đề.\n\nAccuracy (Độ chính xác): 10.0/10.0\nThông tin trong câu trả lời của agent hoàn toàn chính xác so với câu trả lời chuẩn. Agent đã nêu rõ thâm niên được tính từ thời điểm ký HĐLĐ chính thức đầu tiên, không tính hợp đồng thử việc.\n\nCompleteness (Đầy đủ, rõ ràng): 10.0/10.0\nCâu trả lời không chỉ cung cấp thông tin cơ bản như câu trả lời chuẩn mà còn bổ sung nhiều thông tin hữu ích khác như: thâm niên kế thừa, các trường hợp không được tính thâm niên, và ứng dụng của thâm niên trong chính sách công ty. Thông tin bổ sung này rất có giá trị cho người hỏi.\n\nClarity (Rõ ràng, trình bày hợp lý): 10.0/10.0\nCâu trả lời được trình bày rất rõ ràng với cấu trúc logic, sử dụng định dạng đá

In [85]:
zep_evaluate_results[0]

{'scores': {'relevance': 10.0,
  'accuracy': 10.0,
  'completeness': 10.0,
  'clarity': 10.0,
  'tone': 10.0,
  'average': 10.0},
 'comments': "Relevance (10.0/10.0): Câu trả lời hoàn toàn phù hợp với câu hỏi về chính sách thâm niên. Agent đã trả lời đúng trọng tâm câu hỏi về thời điểm bắt đầu tính thâm niên.\n\nAccuracy (10.0/10.0): Thông tin trong câu trả lời của agent hoàn toàn chính xác và khớp với câu trả lời chuẩn. Agent đã nêu rõ 'thâm niên được tính từ thời điểm ký Hợp đồng lao động chính thức đầu tiên'.\n\nCompleteness (10.0/10.0): Câu trả lời không chỉ cung cấp thông tin cơ bản như câu trả lời chuẩn mà còn bổ sung nhiều thông tin hữu ích khác như: chi tiết về cách tính thâm niên, thời gian không được tính vào thâm niên, và ví dụ minh họa cụ thể. Những thông tin bổ sung này rất có giá trị cho người hỏi.\n\nClarity (10.0/10.0): Câu trả lời được trình bày rất rõ ràng, có cấu trúc logic với các đề mục và gạch đầu dòng, giúp người đọc dễ dàng nắm bắt thông tin. Format phân chia rõ

In [ ]:
zep_scores_df = pd.DataFrame({
    "Relevance": [score["scores"]["relevance"] for score in zep_evaluate_results],
    "Accuracy": [score["scores"]["accuracy"] for score in zep_evaluate_results],
    "Completeness": [score["scores"]["completeness"] for score in zep_evaluate_results],
    "Clarity": [score["scores"]["clarity"] for score in zep_evaluate_results],
    "Tone": [score["scores"]["tone"] for score in zep_evaluate_results]}
)

zep_scores_df.to_excel("zep_scores.xlsx", sheet_name="Zep memory", index=False)

default_scores_df = pd.DataFrame({
    "Relevance": [score["scores"]["relevance"] for score in default_evaluate_results],
    "Accuracy": [score["scores"]["accuracy"] for score in default_evaluate_results],
    "Completeness": [score["scores"]["completeness"] for score in default_evaluate_results],
    "Clarity": [score["scores"]["clarity"] for score in default_evaluate_results],
    "Tone": [score["scores"]["tone"] for score in default_evaluate_results]}
)

default_scores_df.to_excel("scores.xlsx", sheet_name="Default memory", index=False)

In [88]:
import pickle
with open("zep_evaluate_results.pkl", "wb") as f:
    pickle.dump(zep_evaluate_results, f)
    
with open("default_evaluate_results.pkl", "wb") as f:
    pickle.dump(default_evaluate_results, f)
    
